# Setup

In [1]:
# Imports
import os
import os.path as osp

import joblib
import numpy as np
import gym
import pandas as pd
import matplotlib.pyplot as plt

import pirl
from pirl import config, experiments

from analysis import common

In [2]:
# Config
experiment_dirs = {
    'classic': 'continuous-baselines-classic-20180627_040027-d00bce3b7702849cc680edbde3e1eee132a19719',
    'easy': 'continuous-baselines-easy-20180627_040107-d00bce3b7702849cc680edbde3e1eee132a19719',
    'medium': 'continuous-baselines-medium-20180628_055905-29490a810bef1f259af20c467a50997fecad6944',
}
experiment_dirs = {k: osp.join('..', 'data', 'aws_live', 'experiments', v) for k, v in experiment_dirs.items()}

# Value difference

In [3]:
algo_pattern = '(.*)'
env_pattern = '(.*)'
dfs = {k: common.load_value(v, algo_pattern, env_pattern)
       for k, v in experiment_dirs.items()}
df = pd.concat(list(dfs.values()), axis=0)
values = common.aggregate_value(df)

/home/adam/bin/anaconda3/envs/mypirl/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [4]:
common.plot_ci(values)

airl random     airl random short                  airl sa            airl sa short              airl so         airl so short  \
env                       n m    eval                                                                                                                                                     
HalfCheetah-v2            0 1000 irl            -4471.238 +/- 21.002           nan +/- nan      1320.587 +/- 23.216              nan +/- nan  -60.591 +/- 120.782           nan +/- nan   
                                 ppo_cts        -1605.537 +/- 57.283           nan +/- nan  -27493.644 +/- 2883.536              nan +/- nan   769.184 +/- 74.742           nan +/- nan   
Hopper-v2                 0 1000 irl                13.719 +/- 2.994           nan +/- nan          3.716 +/- 0.096              nan +/- nan      3.582 +/- 0.099           nan +/- nan   
                                 ppo_cts             4.722 +/- 0.133           nan +/- nan    -5094.845 +/- 895.604              nan +/- nan   315.945 +/- 57.345           nan +/- nan   
InvertedDoublePendulum-v2 0 1000 irl                45.582 +/- 3.140           nan +/- nan     1874.067 +/- 644.735              nan +/- nan     29.811 +/- 1.804           nan +/- nan   
                                 ppo_cts            30.425 +/- 2.510           nan +/- nan       162.728 +/- 20.932              nan +/- nan     28.410 +/- 0.636           nan +/- nan   
InvertedPendulum-v2       0 1000 irl                 4.483 +/- 0.491           nan +/- nan          3.647 +/- 0.106              nan +/- nan      3.000 +/- 0.000           nan +/- nan   
                                 ppo_cts             3.016 +/- 0.025           nan +/- nan       445.163 +/- 78.471              nan +/- nan   540.717 +/- 87.559           nan +/- nan   
MountainCarContinuous-v0  0 1000 irl                     nan +/- nan   -700.086 +/- 22.034              nan +/- nan         89.176 +/- 0.831          nan +/- nan      77.154 +/- 1.099   
                                 ppo_cts_short           nan +/- nan      69.473 +/- 0.795              nan +/- nan  -41065.722 +/- 5120.582          nan +/- nan      60.305 +/- 1.755   
Pendulum-v0               0 1000 irl                     nan +/- nan  -1230.105 +/- 53.325              nan +/- nan     -1109.718 +/- 46.097          nan +/- nan  -1115.262 +/- 35.500   
                                 ppo_cts_short           nan +/- nan  -1547.216 +/- 15.940              nan +/- nan     -1260.444 +/- 40.826          nan +/- nan  -1438.864 +/- 64.729   
Reacher-v2                0 1000 irl             -748.721 +/- 20.995           nan +/- nan         -8.512 +/- 0.920              nan +/- nan    -12.277 +/- 1.584           nan +/- nan   
                                 ppo_cts           -25.983 +/- 4.458           nan +/- nan  -32502.006 +/- 2674.441              nan +/- nan   -132.057 +/- 8.767           nan +/- nan   
Swimmer-v2                0 1000 irl                -0.244 +/- 2.600           nan +/- nan         48.779 +/- 3.424              nan +/- nan     32.935 +/- 5.764           nan +/- nan   
                                 ppo_cts            -4.833 +/- 4.834           nan +/- nan      -400.579 +/- 40.080              nan +/- nan     30.315 +/- 0.901           nan +/- nan   

                                                              expert                  gail            gail short               ppo cts         ppo cts short  
env                       n m    eval                                                                                                                         
HalfCheetah-v2            0 1000 irl            2127.043 +/- 187.089  1894.268 +/- 191.078           nan +/- nan  2127.043 +/- 187.089           nan +/- nan  
                                 ppo_cts        2127.043 +/- 187.089           nan +/- nan           nan +/- nan  2127.043 +/- 187.089           nan +/- nan  
Hopper-v2                 0 1000 irl            1951.9

# Policy rollout

In [ ]:
def expert_cached_value(rl, env_name, pol_discount=0.99, eval_discount=1.00, seed=1234, episodes=100):
    '''Rollout a cached expert policy for episodes.
       WARNING: This will be slow or just break if policy is not in cache!'''
    gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    policy, value = experiments._train_policy(rl, pol_discount, env_name, seed, None)
    vmean, vse = value
    print('Cached value: {:.3f} +/- {:.3f}'.format(vmean, 1.96 * vse))
    
    env = gym.make(env_name)
    rmean, rse = compute_value(env, policy, eval_discount, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(rmean, 1.96 * rse))
    return (vmean, vse), (rmean, rse)

def _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed):
    _gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    fname = osp.join(results_dir, 'policy.pkl')
    print('Loading policy from ', fname)
    policy = joblib.load(fname)
    env = gym.make(env_name)
    
    mean, se = compute_value(env, policy, eval_discount, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))
    return mean, se

def expert_value(experiment_dir, rl, env_name, pol_discount=0.99, eval_discount=1.00, episodes=100, seed=1234):
    results_dir = osp.join(experiment_dir, 'expert', env_name, rl)
    return _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed)

def irl_eval_value(experiment_dir, irl_name, num_traj, rl, env_name, pol_discount=0.99, eval_discount=1.00, episodes=100, seed=1234):
    results_dir = osp.join(experiment_dir, 'eval', env_name, 
                           '{}:{}:{}'.format(irl_name, num_traj, num_traj), rl)
    return _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed)
    
def irl_value(experiment_dir, irl_name, env_name, num_traj, eval_discount=1.00, episodes=100):
    _irl_algo, _reward_wrapper, compute_value = experiments.make_irl_algo(irl_name)
    irl_dir = osp.join(experiment_dir, 'irl', irl_name)
    if not os.path.exists(irl_dir):
        raise FileNotFoundError("No result directory {}".format(irl_dir))
    
    pop_fname = osp.join(irl_dir, str(num_traj), 'policies.pkl')
    sin_fname = osp.join(irl_dir, env_name, str(num_traj), 'policy.pkl')
    if os.path.exists(pop_fname):
        policies = joblib.load(pop_fname)
        print(policies.keys())
        policy = policies[env_name]
    elif os.path.exists(sin_fname):
        policy = joblib.load(sin_fname)
    else:
        raise FileNotFoundError("Neither {} or {} exists".format(pop_fname, sin_fname))
    
    env = gym.make(env_name)
    mean, se = compute_value(env, policy, discount=eval_discount, num_episodes=episodes)
    print('Rollout value: {} +/- {}'.format(mean, 1.96 * se))

In [ ]:
expert_cached_value('ppo_cts', 'Reacher-v2', episodes=100)

In [ ]:
expert_value(experiment_dir, 'ppo_cts', 'Reacher-v2', episodes=500, seed=1234)

In [ ]:
irl_value(experiment_dir, 'airl', 'Reacher-v2', 1000, episodes=100)

# Visualizing rewards (gridworld only)

In [ ]:
def show_heatmaps(irl_algo, kind='inline', out_dir=None, shape=(9,9), **kwargs):
    data = pd.read_pickle(osp.join(experiment_dir, 'results.pkl'))
    rewards = data['rewards'][irl_algo]
    if kind in ['inline', 'pdf']:
        figs = common.gridworld_heatmap(rewards, shape)
        if out_dir is None:
            for fig in figs:
                display(fig[1])
        else:
            common.save_figs(figs, out_dir)
    elif kind == 'movie':
        common.gridworld_heatmap_movie(out_dir, rewards, shape)
    else:
        assert False

In [ ]:
irl_algos = ['mce', 'mcep_reg1e0']
for irl in irl_algos:
    show_heatmaps(irl, kind='inline', shape=(4,4))
    #show_heatmaps(irl, kind='movie', out_dir='figs/jungle/movies/' + irl)
    #show_heatmaps(irl, kind='movie', out_dir='figs/jungle/' + irl)

# Loss curve (PPO only)

In [ ]:
def ppo_progress(results_dir):
    path = osp.join(results_dir, 'progress.csv')
    df = pd.read_csv(path)
    df = df.set_index('serial_timesteps')
    return df

def expert_ppo_progress(experiment_dir, env_name, rl_name):
    results_dir = osp.join(experiment_dir, 'expert',
                       experiments.sanitize_env_name(env_name),
                       rl_name, 'train')
    return ppo_progress(results_dir)

In [ ]:
envs = ['Reacher-v2']
for env in envs:
    df = expert_ppo_progress(osp.join(experiment_dirs['easy'], '0'), env, 'ppo_cts')
    plt.figure()
    df['eprewmean'].plot()
    print(df['eprewmean'].max())
    plt.title(env)

# Evaluate checkpoint

In [ ]:
def load_checkpoint(results_dir, checkpoint_num=None):
    checkpoint_dir = osp.join(results_dir, 'checkpoints')
    if checkpoint_num is None:
        checkpoint_num = max(os.listdir(checkpoint_dir))
    checkpoint_fname = osp.join(checkpoint_dir, checkpoint_num)
    print('Loading from ', checkpoint_fname)
    policy = joblib.load(checkpoint_fname)
    
    return policy

def ppo_value2(results_dir, rl, env_name, pol_discount, eval_discount, episodes):
    _gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    fname = osp.join(results_dir, 'policy.pkl')
    print('Loading policy from ', fname)
    policy = joblib.load(fname)
    env = gym.make(env_name)
    
    mean, se = compute_value(env, policy, eval_discount, num_episodes=episodes)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))
    return mean, se

def ppo_value(policy, env_name, episodes, seed=1234):
    _, _, compute_value = config.RL_ALGORITHMS['ppo_cts']
    env = gym.make(env_name)
    mean, se = compute_value(env, policy, 1.00, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))

In [ ]:
policy = load_checkpoint(osp.join(experiment_dir, 'expert/InvertedPendulum-v2/ppo_cts'), '00488')
ppo_value(policy, 'InvertedPendulum-v2', 500, 1234)